In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

# Loading data
First we have to load data from the various tab-separated files

In [2]:
path = 'data/raw/'
df = pd.DataFrame()
df_list = [pd.read_csv(path + file ,sep='\t',header=None) for file in os.listdir(path)]
df = pd.concat(df_list)


In [3]:
df.columns = ['date', 'time', 'code', 'value']

## Data description
The data set contains a diabetes patient record
Columns:
1) date
2) time
3) code describes (#TODO)
#TODO

In [4]:
df.head()

,date,time,code,value
0,05-20-1991,08:00,58.0,101
1,05-20-1991,08:00,33.0,5
2,05-20-1991,08:00,34.0,27
3,05-20-1991,12:00,60.0,89
4,05-20-1991,12:00,33.0,3
